Utilizando esse conjunto de dados:
https://www.kaggle.com/datasets/disham993/9000-movies-dataset

Responda as seguintes perguntas:

1.   Qual tamanho do DataSet?
2.   Quantas linhas?
3.   Quantas colunas?
4.   Qual o tipo de variável de cada coluna?
5.   Qual o filme com maior número de votaçôes?
6.   Qual filme teve a maior nota (critério de desempate é o filme com mais votos)
7.   Existem valores nulos? Se sim, qual tratamento irá realizar? (Se não temos nome de algum filme, melhor nem considerar)
8.   Transforme as variaveis categóricas de linguagem e genero em númericas (utilize dummy)
9.   Normalize as variaveis numéricas
10.  Armazene esses valores como um artefato dentro do MLFlow
11. QUais insights é possivel obter desses dados?



In [ ]:
import pandas as pd
import numpy as np

# Gerenciamento de experimentos com MLflow
import mlflow

# Download do dataset de filmes do Kaggle
#!kaggle datasets download -d disham993/9000-movies-dataset -p ./movies_dataset

# Descompactar o arquivo baixado
#!unzip -o ./movies_dataset/9000-movies-dataset.zip -d ./movies_dataset


# Carregar o dataset de filmes usando o motor 'python' para evitar o ParserError
try:
    df = pd.read_csv("./movies_dataset/mymoviedb.csv", engine='python')
    print("O dataset foi importado com sucesso!")

    # Obter o número de linhas e colunas do DataFrame
    numero_linhas, numero_colunas = df.shape
    print(f"O dataset tem {numero_linhas} linhas")
    print(f"O dataset tem {numero_colunas} colunas.")

    colunas_object= df.select_dtypes(include=['object'])
    colunas_int64 = df.select_dtypes(include=['int64'])
    colunas_outrotipo = df.select_dtypes(exclude=['object', 'int64'])
    print(f"As colunas do tipo 'object' são: {colunas_object.columns.tolist()}")
    print(f"As colunas do tipo 'int64' são: {colunas_int64.columns.tolist()}")
    print(f"As colunas de outro tipo são: {colunas_outrotipo.columns.tolist()}")
    
except Exception as e:
    print(f"Ocorreu um erro ao carregar o CSV: {e}")
    print("Verifique se o arquivo 'mymoviedb.csv' existe no diretório './movies_dataset' e não está corrompido.")
